In [ ]:
import numpy as np
import chess
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

from datetime import datetime
import warnings
import pickle
import json
import os
warnings.filterwarnings("ignore")

# Let's Play

In [ ]:
def get_board_vector(board, flip=False):
    vec = np.zeros((64,), dtype="float32")
    curr_player = board.turn
    flip_mult = 1 if not flip else -1
    for pos in range(64):
        piece = board.piece_at(pos)
        if piece:
            if piece.color == curr_player:
                piece_mult = 1
            else:
                piece_mult = -1
            vec[pos] = flip_mult*piece_mult*int(piece.piece_type)
    return vec


def make_move(board, move, piece_value_map, promotion_value_map):
    """Return a new board vector and reward and whether it finishes the game
    Assume the move is already validated and legal"""
    reward = 0
    done = False
    init_pos, new_pos, promotion = move[:2], move[2:4], move[4:5]
    if board.piece_at(chess.SQUARE_NAMES.index(new_pos)):
        reward += piece_value_map[board.piece_at(
            chess.SQUARE_NAMES.index(new_pos)).piece_type]
    if promotion:
        reward += promotion_value_map[promotion]
    board.push(chess.Move.from_uci(move))
    new_state = get_board_vector(board, flip=True)
    if board.is_game_over():
        done = True
    return new_state, reward, done

def neural_network_model(input_size, output_size, learning_rate=.0001):
    network = input_data(shape=[None, input_size], name='input')
    network = fully_connected(network, 128, activation='relu')
    network = fully_connected(network, 128, activation='relu')
    network = fully_connected(network, output_size, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=learning_rate,
                         loss='mean_square', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')
    return model

In [ ]:
def model_turn(model):
    board_vec = get_board_vector(board)
    action_ls = model.predict(board_vec.reshape(1, -1))[0]
    sorted_indicies = np.flip(np.argsort(action_ls))
    move = None
    curr_ind = 0
    while not move:
        move = chess.Move.from_uci(action_map[str(sorted_indicies[curr_ind])])
        if move not in board.legal_moves:
            curr_ind += 1
            move = None
    board.push(move)
    print("Move Rank: ", curr_ind)
    
def player_turn(move_string):
    board.push(chess.Move.from_uci(move_string))

In [ ]:
model_path=''

In [ ]:
with open('../action_map.json') as fp:
    action_map = json.load(fp)
trained_model = neural_network_model(input_size=64,
                                  output_size=len(action_map))

In [ ]:
trained_model.load(model_path)

In [ ]:
board = chess.Board()
board

In [ ]:
player_turn('e2e4')
model_turn(trained_model)
board